In [10]:


import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.animation as animation
import time
import random
from collections import defaultdict

# --- ---

RUN_WITH_ANIMATION = False

SIMULATION_SETTINGS = {
    "dt": 0.1,
    "targeting_strategy": 'NEAREST',
    "is_coordinated": True
}
WORLD_SETTINGS = {
    'width': 1000,
    'height': 1000
}
BATTLEFIELD_UNITS = [
    {"id": "VSHORAD_1", "type": "C-RAM", "position": (400, 500)},
    {"id": "VSHORAD_2", "type": "C-RAM", "position": (400, 700)},
    {"id": "Wisla IBCS", "type": "LASER", "position": (600, 600)}
]
SYSTEM_TYPES = {
    "C-RAM": {
        "range": 250, "channels": 1, "service_time": 0.5,
        "damage_per_hit": 1, "magazine_size": 150, "reload_time": 600.0,
        "color": 'blue'
    },
    "LASER": {
        "range": 400, "channels": 2, "service_time": 3.0,
        "damage_per_hit": 3, "magazine_size": 12, "reload_time": 10.0,
        "color": 'red'
    }
}
DRONE_TYPES = {
    "Shahed-131": {
        "speed": 60, "hp": 1, "sway_strength": 20.0, "color": 'gray'
    },
    "Shahed-136": {
        "speed": 35, "hp": 3, "sway_strength": 10.0, "color": 'black'
    }
}
ROUTES = {
    "Route_North": { "start": (500, 1000), "target": (500, 0) },
    "Route_West": { "start": (250, 1000), "target": (500, 0) },
    "Route_East": { "start": (750, 1000), "target": (500, 0) }
}
SPAWN_WAVE = [
    {"route": "Route_North", "type": "Shahed-136", "count": 20, "interval": 0},
    {"route": "Route_West", "type": "Shahed-131", "count": 10, "interval": 0},
    {"route": "Route_East", "type": "Shahed-131", "count": 10, "interval": 0},
    {"route": "Route_North", "type": "Shahed-136", "count": 10, "interval": 0}
]




In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import time
import copy 

from simulation import Simulation, WORLD_SETTINGS, SIMULATION_SETTINGS, \
                              SYSTEM_TYPES, DRONE_TYPES, ROUTES, SPAWN_WAVE


POPULATION_SIZE = 20


NUM_GENERATIONS = 30

MUTATION_RATE = 0.2

MUTATION_STRENGTH = 150.0

N_EVALS_PER_CHROMOSOME = 5

ELITISM_COUNT = 5

# --- Definicje struktur GA ---

class Individual:
    def __init__(self, units_layout):
        self.layout = units_layout
        self.fitness = 0.0 
        self.avg_leaked = float('inf') 

    def __repr__(self):
        return f"[Fitness: {self.fitness:.4f} | Avg Leaked: {self.avg_leaked:.2f}]"

def create_random_individual():
    original_units = copy.deepcopy(BATTLEFIELD_UNITS) 
    
    new_layout = []
    for unit in original_units:
        new_x = random.uniform(0, WORLD_SETTINGS['width'])
        new_y = random.uniform(0, WORLD_SETTINGS['height'])
        
        new_unit = {
            "id": unit['id'],
            "type": unit['type'],
            "position": (new_x, new_y) 
        }
        new_layout.append(new_unit)
        
    return Individual(new_layout)

def calculate_fitness(individual, n_runs=N_EVALS_PER_CHROMOSOME):

    leaked_results = []
    sim_settings = copy.deepcopy(SIMULATION_SETTINGS)
    sim_wave = copy.deepcopy(SPAWN_WAVE)
    
    for _ in range(n_runs):
        
        sim = Simulation(
            world=WORLD_SETTINGS,
            sim_settings=sim_settings,
            system_types=SYSTEM_TYPES,
            units=individual.layout, 
            drone_types=DRONE_TYPES,
            routes=ROUTES,
            wave=sim_wave
        )
        
        while not sim.is_finished():
            sim.step()
            
        leaked_results.append(len(sim.leaked_drones))
    
    avg_leaked = np.mean(leaked_results)
    
    individual.avg_leaked = avg_leaked
    individual.fitness = 1.0 / (1.0 + avg_leaked)


def crossover(parent1, parent2):

    child_layout = []
    
    num_units = len(parent1.layout)
    
    for i in range(num_units):
        if random.random() < 0.5:
            child_layout.append(copy.deepcopy(parent1.layout[i]))
        else:
            child_layout.append(copy.deepcopy(parent2.layout[i]))
            
    return Individual(child_layout)

def mutate(individual, rate, strength):
    if random.random() > rate:
        return 
        
    unit_to_mutate = random.choice(individual.layout)
    old_x, old_y = unit_to_mutate['position']
    new_x = old_x + random.uniform(-strength, strength)
    new_y = old_y + random.uniform(-strength, strength)
    
    new_x = np.clip(new_x, 0, WORLD_SETTINGS['width'])
    new_y = np.clip(new_y, 0, WORLD_SETTINGS['height'])
    
    unit_to_mutate['position'] = (new_x, new_y)

def select_parent(population):

    p1 = random.choice(population)
    p2 = random.choice(population)
    return p1 if p1.fitness > p2.fitness else p2

def plot_final_placement(individual, generation):

    sim = Simulation(
        world=WORLD_SETTINGS,
        sim_settings=SIMULATION_SETTINGS,
        system_types=SYSTEM_TYPES,
        units=individual.layout,
        drone_types=DRONE_TYPES,
        routes=ROUTES,
        wave=[] 
    )
    
    from simulation import draw_static_environment, draw_systems

    fig, ax = plt.subplots(figsize=(10, 10))
    draw_static_environment(ax, sim)
    draw_systems(ax, sim) 
    
    ax.set_title(f"Zoptymalizowane Rozmieszczenie (Pokolenie {generation})\n"
                 f"Minimalny średni przeciek: {individual.avg_leaked:.2f} dronów")
    plt.show()

def main():
    print("--- Rozpoczynanie Optymalizacji Algorytmem Genetycznym ---")

    total_time_start = time.time()
    best_individual_so_far = None
    population = [create_random_individual() for _ in range(POPULATION_SIZE)]

    for gen in range(NUM_GENERATIONS):
        gen_time_start = time.time()
        print(f"\n--- Pokolenie {gen + 1} / {NUM_GENERATIONS} ---")
        
        for ind in population:
            calculate_fitness(ind)
            
        population.sort(key=lambda x: x.fitness, reverse=True)
        current_best = population[0]
        if best_individual_so_far is None or current_best.fitness > best_individual_so_far.fitness:
            best_individual_so_far = copy.deepcopy(current_best)

        gen_time_end = time.time()
        print(f"Najlepszy w tym pokoleniu: {current_best}")
        print(f"Najlepszy w historii: {best_individual_so_far}")
        print(f"Czas pokolenia: {gen_time_end - gen_time_start:.2f}s")
        
        if gen == NUM_GENERATIONS - 1:
            break 
        new_population = []
        new_population.extend(population[:ELITISM_COUNT])
        
        fill_count = POPULATION_SIZE - ELITISM_COUNT
        for _ in range(fill_count):
            parent1 = select_parent(population)
            parent2 = select_parent(population)
            child = crossover(parent1, parent2)
            
            mutate(child, MUTATION_RATE, MUTATION_STRENGTH)
            
            new_population.append(child)
            
        population = new_population

    total_time_end = time.time()
    print("\n\n--- OPTYMALIZACJA ZAKOŃCZONA ---")
    print(f"Całkowity czas: {total_time_end - total_time_start:.2f}s")
    print("Najlepsze znalezione rozmieszczenie:")
    print(f"  Średni przeciek: {best_individual_so_far.avg_leaked:.3f} dronów")
    print("  Układ jednostek:")
    for unit in best_individual_so_far.layout:
        pos = unit['position']
        print(f"    - {unit['id']} ({unit['type']}): ({pos[0]:.0f}, {pos[1]:.0f})")

    plot_final_placement(best_individual_so_far, NUM_GENERATIONS)

if __name__ == "__main__":
    main()

--- Rozpoczynanie Optymalizacji Algorytmem Genetycznym ---

--- Pokolenie 1 / 30 ---
Ocenianie 20 osobników...
Najlepszy w tym pokoleniu: [Fitness: 0.0746 | Avg Leaked: 12.40]
Najlepszy w historii: [Fitness: 0.0746 | Avg Leaked: 12.40]
Czas pokolenia: 4.25s

--- Pokolenie 2 / 30 ---
Ocenianie 20 osobników...
Najlepszy w tym pokoleniu: [Fitness: 0.0725 | Avg Leaked: 12.80]
Najlepszy w historii: [Fitness: 0.0746 | Avg Leaked: 12.40]
Czas pokolenia: 3.74s

--- Pokolenie 3 / 30 ---
Ocenianie 20 osobników...
Najlepszy w tym pokoleniu: [Fitness: 0.0909 | Avg Leaked: 10.00]
Najlepszy w historii: [Fitness: 0.0909 | Avg Leaked: 10.00]
Czas pokolenia: 3.48s

--- Pokolenie 4 / 30 ---
Ocenianie 20 osobników...
Najlepszy w tym pokoleniu: [Fitness: 0.0980 | Avg Leaked: 9.20]
Najlepszy w historii: [Fitness: 0.0980 | Avg Leaked: 9.20]
Czas pokolenia: 3.63s

--- Pokolenie 5 / 30 ---
Ocenianie 20 osobników...
Najlepszy w tym pokoleniu: [Fitness: 0.1000 | Avg Leaked: 9.00]
Najlepszy w historii: [Fitness: